# 誤差逆伝播法に対応したNNを実装する

In [34]:
import sys, os
import numpy as np
from collections import OrderedDict
from common.layers import SoftmaxWithLoss, Affine, ReLU

### [演習]
* 以下の誤差逆伝播法に対応したNNクラスを完成させましょう。

In [35]:
# ヒント

# OrderedDictを用いると、順序を保持したdictを生成することができる
# ただし、Pythonバージョン3.6以降は、普通のdictでも順序が保存される
dic1 = OrderedDict()
dic1["a"] = 1
dic1["c"] = 3
dic1["b"] = 2
for key, value in dic1.items():
    print(key, value)
print()

# dictから逆順でvalueを取り出す方法
layers = list(dic1.values())
layers.reverse() # 順序を逆転させる
print(layers)

a 1
c 3
b 2

[2, 3, 1]


In [36]:
class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # 重みの初期化
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) 
        self.params['b2'] = np.zeros(output_size)

        # レイヤの生成
        self.layers = OrderedDict() # 順番付きdict形式. 
        self.layers['Affine1'] = Affine( self.params["W1"]  ,self.params["b1"]   )
        self.layers['Relu1'] = ReLU()
        self.layers['Affine2'] = Affine(self.params["W2"]    , self.params["b2"] )
        self.lastLayer = SoftmaxWithLoss() # 出力層
        
    def predict(self, x):
        """
        予測関数
        x : 入力
        """
        for layer in self.layers.values():
            # 入力されたxを更新していく = 順伝播計算
            x = layer.forward(x)
        
        return x
        
    def loss(self, x, t):
        """
        損失関数
        x:入力データ, t:正解データ
        """
        y = self.predict(x)
        return self.lastLayer.forward(y,t)
    
    def accuracy(self, x, t):
        """
        正解率
        """
        # 予測. 返り値は正規化されていない実数
        y = self.predict(x)
        
        #正規化されていない実数をもとに、最大値になるindexに変換する
        y = np.argmax(y, axis=1)
        
        if t.ndim != 1 : 
            """
            one-hotベクトルの場合、正解データをindexに変換する
            """
            t = np.argmax(t, axis=1  )
        
        # 正解率
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    def gradient(self, x, t):
        """
        全パラメータの勾配を計算
        """
        
        # 順伝播
        self.loss(x, t)

        # 逆伝播
        dout = self.lastLayer.backward() # 出力層
        
        ## doutを逆向きに伝える 
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # dW, dbをgradsにまとめる
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW,self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW,self.layers['Affine2'].db

        return grads

In [37]:
# 学習用データ
x = np.array([[1, 2, 3],
            [3, 2, 1]])
t = np.array([[0, 0, 1],
            [1, 0, 0]])

tnet = TwoLayerNet( input_size=3, hidden_size=5, output_size=3)

print("predict=\n", tnet.predict(x))
print()
print("loss=\n", tnet.loss(x, t))
print()
print("accuracy=", tnet.accuracy(x, t))
print()
print("gradient=\n",)
grads = tnet.gradient(x, t)
for key, value in grads.items():
    print(key)
    print(value)

predict=
 [[ 0.00012945  0.00077276 -0.00028625]
 [ 0.00035459  0.000959   -0.00040846]]

loss=
 1.0988314607519616

accuracy= 0.0

gradient=

W1
[[ 0.          0.00703071 -0.00517373 -0.01578026  0.        ]
 [ 0.          0.00707509  0.00825996 -0.01052018  0.        ]
 [ 0.          0.00711946  0.02169365 -0.00526009  0.        ]]
b1
[ 0.          0.00353754  0.00412998 -0.00526009  0.        ]
W2
[[ 0.          0.          0.        ]
 [-0.01209314  0.02167049 -0.00957735]
 [-0.00434935  0.00599442 -0.00164507]
 [-0.00304019  0.00152113  0.00151905]
 [ 0.          0.          0.        ]]
b2
[-0.16667054  0.33353748 -0.16686694]
